<a href="https://colab.research.google.com/github/EmperoR1127/ml/blob/master/csi5155_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images/"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

data = arff.loadarff('/content/drive/My Drive/Data/seismic-bumps.arff')
seismic_bumps_data = pd.DataFrame(data[0])
#pre-process the train_set
train_labels = seismic_bumps_data[["class"]].copy()
train_set = seismic_bumps_data.drop(["class"], axis=1)
train_labels["class"] = train_labels["class"].map(lambda x: str(x)[2])
train_set_num = seismic_bumps_data.drop(["seismic","seismoacoustic","shift", "ghazard", "class"], axis=1)
train_set_cat = seismic_bumps_data.drop(["genergy","gpuls","gdenergy", "gdpuls", "nbumps", "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"], axis=1)
#build the pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
full_pipeline = ColumnTransformer([("num", num_pipeline, list(train_set_num)),("cat", OneHotEncoder(), list(train_set_cat)),])
#prepare the data
train_set_prepared = full_pipeline.fit_transform(train_set)
#prepare the target
encoder = LabelEncoder()
train_labels_prepared = encoder.fit_transform(train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
#rebalance the dataset using random oversampling
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=42)
ros_train_set_prepared, ros_train_labels_prepared = ros.fit_resample(train_set_prepared, train_labels_prepared)
print(sorted(Counter(ros_train_labels_prepared).items()))

[(0, 2414), (1, 2414)]


In [14]:
#rebalance the dataset using undersampling (nearest neightbours)
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours()
renn_train_set_prepared, renn_train_labels_prepared = renn.fit_resample(train_set_prepared, train_labels_prepared)
print(sorted(Counter(renn_train_labels_prepared).items()))


[(0, 1920), (1, 170)]


In [15]:
#rebalance the dataset using balanced sampling (SMOTEENN)
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
smote_enn_train_set_prepared, smote_enn_train_labels_prepared = smote_enn.fit_resample(train_set_prepared, train_labels_prepared)
print(sorted(Counter(smote_enn_train_labels_prepared).items()))

[(0, 1818), (1, 2194)]


In [20]:
#train the model using 10-fold cross validation and get the confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#stores cunfustion matrices of model training on 3 different datasets
confusion_matrix_dict = dict()
dt_clf = DecisionTreeClassifier()
#train the model on the oversampling dataset
dt_ros_train_prediction = cross_val_predict(dt_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
confusion_matrix_dict["oversampling confusion matrix"] = confusion_matrix(ros_train_labels_prepared, dt_ros_train_prediction)
#train the model on the undersampling dataset
dt_renn_train_prediction = cross_val_predict(dt_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
confusion_matrix_dict["undersampling confusion matrix"] = confusion_matrix(renn_train_labels_prepared, dt_renn_train_prediction)
#train the model on the balanced sampling dataset
dt_smote_enn_train_prediction = cross_val_predict(dt_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)
confusion_matrix_dict["balanced sampling confusion matrix"] = confusion_matrix(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
confusion_matrix_dict

{'balanced sampling confusion matrix': array([[1574,  244],
        [ 106, 2088]]), 'oversampling confusion matrix': array([[2156,  258],
        [   0, 2414]]), 'undersampling confusion matrix': array([[1734,  186],
        [ 102,   68]])}